# 学习

## 导入包

In [6]:
# -*- coding: utf-8 -*-
# 使用pandas 读取上面介绍的数据，并进行简单的缺失值填充
import pandas as pd
from sklearn.metrics import log_loss, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from deepctr_torch.models import *
from deepctr_torch.inputs import SparseFeat, DenseFeat, get_feature_names
import torch

## 导入数据

In [12]:
# 使用pandas 读取上面介绍的数据，并进行简单的缺失值填充
data = pd.read_csv('./criteo_sample.txt')
data


,label,I1,I2,I3,I4,I5,I6,I7,I8,I9,...,C17,C18,C19,C20,C21,C22,C23,C24,C25,C26
0,0,NaN,3,260.0,NaN,17668.0,NaN,NaN,33.0,NaN,...,e5ba7672,87c6f83c,NaN,NaN,0429f84b,NaN,3a171ecb,c0d61a5c,NaN,NaN
1,0,NaN,-1,19.0,35.0,30251.0,247.0,1.0,35.0,160.0,...,d4bb7bd8,6fc84bfb,NaN,NaN,5155d8a3,NaN,be7c41b4,ded4aac9,NaN,NaN
2,0,0.0,0,2.0,12.0,2013.0,164.0,6.0,35.0,523.0,...,e5ba7672,675c9258,NaN,NaN,2e01979f,NaN,bcdee96c,6d5d1302,NaN,NaN
3,0,NaN,13,1.0,4.0,16836.0,200.0,5.0,4.0,29.0,...,e5ba7672,52e44668,NaN,NaN,e587c466,NaN,32c7478e,3b183c5c,NaN,NaN
4,0,0.0,0,104.0,27.0,1990.0,142.0,4.0,32.0,37.0,...,e5ba7672,25c88e42,21ddcdc9,b1252a9d,0e8585d2,NaN,32c7478e,0d4a6d1a,001f3601,92c878de
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,0,NaN,0,113.0,3.0,3036.0,575.0,2.0,3.0,214.0,...,07c540c4,9880032b,21ddcdc9,5840adea,34cc61bb,c9d4222a,32c7478e,e5ed7da2,ea9a246c,984e0db0
196,1,0.0,1,1.0,1.0,1607.0,12.0,1.0,12.0,15.0,...,1e88c74f,3972b4ed,NaN,NaN,d1aa4512,NaN,32c7478e,9257f75f,NaN,NaN
197,1,1.0,0,6.0,3.0,0.0,0.0,19.0,3.0,3.0,...,3486227d,5aed7436,54591762,a458ea53,4a2c3526,NaN,32c7478e,1793a828,e8b83407,1a02cbe1
198,0,0.0,22,6.0,22.0,203.0,153.0,80.0,18.0,508.0,...,3486227d,13145934,55dd3565,5840adea,bf647035,NaN,32c7478e,1481ceb4,e8b83407,988b0775


In [14]:
sparse_features = ['C' + str(i) for i in range(1, 27)]
dense_features = ['I' + str(i) for i in range(1, 14)]

data[sparse_features] = data[sparse_features].fillna('-1', )#将C1-c26列的空值替换为-1
data[dense_features] = data[dense_features].fillna(0, )#将I1-I14列的空值替换为0
target = ['label']



In [13]:
data

,label,I1,I2,I3,I4,I5,I6,I7,I8,I9,...,C17,C18,C19,C20,C21,C22,C23,C24,C25,C26
0,0,NaN,3,260.0,NaN,17668.0,NaN,NaN,33.0,NaN,...,e5ba7672,87c6f83c,NaN,NaN,0429f84b,NaN,3a171ecb,c0d61a5c,NaN,NaN
1,0,NaN,-1,19.0,35.0,30251.0,247.0,1.0,35.0,160.0,...,d4bb7bd8,6fc84bfb,NaN,NaN,5155d8a3,NaN,be7c41b4,ded4aac9,NaN,NaN
2,0,0.0,0,2.0,12.0,2013.0,164.0,6.0,35.0,523.0,...,e5ba7672,675c9258,NaN,NaN,2e01979f,NaN,bcdee96c,6d5d1302,NaN,NaN
3,0,NaN,13,1.0,4.0,16836.0,200.0,5.0,4.0,29.0,...,e5ba7672,52e44668,NaN,NaN,e587c466,NaN,32c7478e,3b183c5c,NaN,NaN
4,0,0.0,0,104.0,27.0,1990.0,142.0,4.0,32.0,37.0,...,e5ba7672,25c88e42,21ddcdc9,b1252a9d,0e8585d2,NaN,32c7478e,0d4a6d1a,001f3601,92c878de
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,0,NaN,0,113.0,3.0,3036.0,575.0,2.0,3.0,214.0,...,07c540c4,9880032b,21ddcdc9,5840adea,34cc61bb,c9d4222a,32c7478e,e5ed7da2,ea9a246c,984e0db0
196,1,0.0,1,1.0,1.0,1607.0,12.0,1.0,12.0,15.0,...,1e88c74f,3972b4ed,NaN,NaN,d1aa4512,NaN,32c7478e,9257f75f,NaN,NaN
197,1,1.0,0,6.0,3.0,0.0,0.0,19.0,3.0,3.0,...,3486227d,5aed7436,54591762,a458ea53,4a2c3526,NaN,32c7478e,1793a828,e8b83407,1a02cbe1
198,0,0.0,22,6.0,22.0,203.0,153.0,80.0,18.0,508.0,...,3486227d,13145934,55dd3565,5840adea,bf647035,NaN,32c7478e,1481ceb4,e8b83407,988b0775


## 简单的预处理 

通常有两种简单的方法来编码用于嵌入的稀疏分类特征（将特征编码后用于嵌入）

这里使用标签编码：将特征映射到从 0 ~ len(#unique) - 1 的整数值，即数量-1

In [48]:
#离散数据的编码
for feat in sparse_features:
    lbe = LabelEncoder()
    data[feat] = lbe.fit_transform(data[feat])
    

In [49]:
data[sparse_features]

,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,...,C17,C18,C19,C20,C21,C22,C23,C24,C25,C26
0,0,4,96,146,1,4,163,1,1,72,...,8,66,0,0,3,0,1,96,0,0
1,11,1,98,98,1,6,179,0,1,89,...,7,52,0,0,47,0,7,112,0,0
2,0,18,39,52,3,4,140,2,1,93,...,8,49,0,0,25,0,6,53,0,0
3,0,45,7,117,1,0,164,1,0,20,...,8,37,0,0,156,0,0,32,0,0
4,0,11,59,77,1,5,18,1,1,45,...,8,14,5,3,9,0,0,5,1,47
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,0,0,66,107,1,4,10,15,1,11,...,0,74,5,1,30,5,0,118,17,48
196,21,64,44,65,1,6,4,2,1,9,...,1,25,0,0,138,0,0,68,0,0
197,0,5,153,143,1,0,141,1,1,118,...,4,40,17,2,41,0,0,12,16,11
198,0,84,84,41,5,4,27,1,1,96,...,4,7,18,1,123,0,0,10,16,49


而对于密集的数值特征，它们通常被离散到桶中，这里我们使用归一化。

In [50]:
mms = MinMaxScaler(feature_range=(0,1))#最大值最小值归一化函数
data[dense_features] = mms.fit_transform(data[dense_features])#连续值的归一化


In [51]:
data[dense_features]

,I1,I2,I3,I4,I5,I6,I7,I8,I9,I10,I11,I12,I13
0,0.000000,0.001332,0.092362,0.000000,0.034825,0.000000,0.000000,0.673469,0.000000,0.000000,0.00000,0.0,0.000000
1,0.000000,0.000000,0.006750,0.402299,0.059628,0.117284,0.003322,0.714286,0.154739,0.000000,0.03125,0.0,0.343137
2,0.000000,0.000333,0.000710,0.137931,0.003968,0.077873,0.019934,0.714286,0.505803,0.000000,0.09375,0.0,0.176471
3,0.000000,0.004664,0.000355,0.045977,0.033185,0.094967,0.016611,0.081633,0.028046,0.000000,0.06250,0.0,0.039216
4,0.000000,0.000333,0.036945,0.310345,0.003922,0.067426,0.013289,0.653061,0.035783,0.000000,0.03125,0.0,0.264706
...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,0.000000,0.000333,0.040142,0.034483,0.005984,0.273029,0.006645,0.061224,0.206963,0.000000,0.03125,0.0,0.029412
196,0.000000,0.000666,0.000355,0.011494,0.003168,0.005698,0.003322,0.244898,0.014507,0.000000,0.03125,0.0,0.117647
197,0.027027,0.000333,0.002131,0.034483,0.000000,0.000000,0.063123,0.061224,0.002901,0.333333,0.28125,0.0,0.000000
198,0.000000,0.007662,0.002131,0.252874,0.000400,0.072650,0.265781,0.367347,0.491296,0.000000,0.34375,0.0,0.215686


## 生成特征列

对于离散型特征，通过嵌入技术将它们转换为密集向量。对于密集的连续型数值特征，我们将它们连接到全连接层的输入张量。
- 标签编码

In [52]:
fixlen_feature_columns = [SparseFeat(feat, vocabulary_size=data[feat].nunique(),embedding_dim=4)
                       for i,feat in enumerate(sparse_features)] + [DenseFeat(feat, 1,)
                       for feat in dense_features]
                       

- 生成特征列

In [54]:
dnn_feature_columns = fixlen_feature_columns
linear_feature_columns = fixlen_feature_columns

feature_names = get_feature_names(linear_feature_columns + dnn_feature_columns)#所有特征列的名字

        

## 生成训练样本并训练模型

- 生成训练样本

In [55]:
train, test = train_test_split(data, test_size=0.2, random_state=2020)
train_model_input = {name: train[name] for name in feature_names}
test_model_input = {name: test[name] for name in feature_names}

In [56]:
train_model_input

{'C1': 76     19
 34      0
 1      11
 40     24
 151    25
        ..
 195     0
 118    11
 67      0
 136     9
 96      0
 Name: C1, Length: 160, dtype: int64,
 'C2': 76     36
 34     56
 1       1
 40     25
 151    50
        ..
 195     0
 118    18
 67     22
 136    18
 96     19
 Name: C2, Length: 160, dtype: int64,
 'C3': 76     146
 34      72
 1       98
 40      34
 151    160
       ... 
 195     66
 118      0
 67      78
 136      0
 96      97
 Name: C3, Length: 160, dtype: int64,
 'C4': 76     117
 34       1
 1       98
 40      62
 151      6
       ... 
 195    107
 118      0
 67     110
 136      0
 96     109
 Name: C4, Length: 160, dtype: int64,
 'C5': 76     4
 34     1
 1      1
 40     6
 151    1
       ..
 195    1
 118    5
 67     0
 136    6
 96     4
 Name: C5, Length: 160, dtype: int64,
 'C6': 76     0
 34     4
 1      6
 40     3
 151    2
       ..
 195    4
 118    0
 67     4
 136    4
 96     5
 Name: C6, Length: 160, dtype: int64,
 'C7': 76 

In [57]:
train, test = train_test_split(data, test_size=0.2)
train_model_input = {name:train[name].values for name in feature_names}
test_model_input = {name:test[name].values for name in feature_names}

In [58]:
train_model_input

{'C1': array([ 1, 20,  0, 11,  9,  0,  0, 21, 25,  6, 14, 10, 11, 11,  0, 16,  0,
        18,  9, 11,  0,  0,  0, 16,  6,  0,  0, 25,  0, 11, 11, 16,  1, 16,
        11,  0, 16,  0,  0, 11, 24, 14,  0, 16,  0, 11,  0,  0, 10, 13,  7,
         0, 11,  0, 10, 18, 16, 12,  0,  0, 11,  0, 11, 10,  0,  0,  9, 11,
         9,  0, 11, 17,  0,  9,  0,  9, 11,  0, 16,  6, 11,  8, 26, 21, 21,
        21, 11, 19,  2,  0, 11,  0,  9,  0, 23,  0,  6,  0,  0,  0,  7, 11,
         0, 11,  0,  0,  0, 18,  0, 21, 11, 11,  6,  0, 19, 11,  0,  0, 16,
         0, 11, 15, 21,  0, 16,  0, 16,  9,  0,  0,  9, 10,  0,  0, 11,  0,
         0, 16, 16,  0,  3,  0,  0,  0, 11,  0, 22,  9,  0, 11, 11,  0,  0,
        11,  0,  0,  0,  9, 21,  0], dtype=int64),
 'C2': array([30, 54, 44, 44, 73,  9, 72, 66, 18, 72, 18, 12, 79, 18, 18, 30, 19,
        10, 60,  5, 46,  4, 52, 82, 71, 55, 23, 50, 34, 84, 17,  5, 13,  5,
        18, 57,  7, 84,  0, 13, 25, 40, 35,  0,  0, 24, 35, 23, 30, 16, 31,
        46, 61, 18, 55, 1

In [61]:
device = 'cpu'
use_cuda = True
if use_cuda and torch.cuda.is_available():
    print('cuda ready...')
    device = 'cuda:0'

model = DeepFM(linear_feature_columns,dnn_feature_columns,task='binary',device=device)

model.compile("adagrad", "binary_crossentropy",
                  metrics=["binary_crossentropy", "auc"], )
history = model.fit(train_model_input,train[target].values,batch_size=256,epochs=50,verbose=2,validation_split=0.2)
pred_ans = model.predict(test_model_input, batch_size=256)

train, test = train_test_split(data, test_size=0.2)

print("")
print("test LogLoss", round(log_loss(test[target].values, pred_ans), 4))
print("test AUC", round(roc_auc_score(test[target].values, pred_ans), 4))




cpu
Train on 128 samples, validate on 32 samples, 1 steps per epoch
Epoch 1/50
0s - loss:  0.6900 - binary_crossentropy:  0.6900 - auc:  0.4810 - val_binary_crossentropy:  0.5954 - val_auc:  0.5407
Epoch 2/50
0s - loss:  0.5902 - binary_crossentropy:  0.5902 - auc:  0.9997 - val_binary_crossentropy:  0.5269 - val_auc:  0.6741
Epoch 3/50
0s - loss:  0.5026 - binary_crossentropy:  0.5026 - auc:  1.0000 - val_binary_crossentropy:  0.4835 - val_auc:  0.6370
Epoch 4/50
0s - loss:  0.4405 - binary_crossentropy:  0.4405 - auc:  1.0000 - val_binary_crossentropy:  0.4682 - val_auc:  0.6296
Epoch 5/50
0s - loss:  0.3816 - binary_crossentropy:  0.3816 - auc:  1.0000 - val_binary_crossentropy:  0.4589 - val_auc:  0.5852
Epoch 6/50
0s - loss:  0.3203 - binary_crossentropy:  0.3203 - auc:  1.0000 - val_binary_crossentropy:  0.4511 - val_auc:  0.5481
Epoch 7/50
0s - loss:  0.2588 - binary_crossentropy:  0.2588 - auc:  1.0000 - val_binary_crossentropy:  0.4464 - val_auc:  0.5333
Epoch 8/50
0s - loss: 